In [26]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [27]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [28]:
tf.keras.backend.clear_session()

In [29]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [30]:
subject = 'Espírito Santo - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [31]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_ES.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data = data.drop('Espírito Santo - Produção de Cimento (t)', axis=1)
data

,Ano,Espírito Santo - IDH Educacao,Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - Desemprego,Espírito Santo - IDH Renda,Espírito Santo - IDH Longevidade,Contratação Comercial (Bi R$),...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Espírito Santo - IDH,Espírito Santo - Consumo de Cimento (t)
0,2008,0.712893,6.711793e+07,3.666068e+06,16.994002,6.044984e+07,7.934578,0.738517,0.861639,5.483335e+08,...,11.760983,21.747715,1015.304010,1377.758492,24.341274,19.583877,76.458036,1.982089,0.768383,1263.713000
1,2009,0.715558,6.738295e+07,3.683277e+06,16.935445,6.029388e+07,7.964300,0.737559,0.864109,5.302764e+08,...,9.472585,22.032901,1015.268312,1389.444333,24.312317,19.520664,76.321047,1.962288,0.769638,1138.824000
2,2010,0.718108,6.901541e+07,3.673390e+06,16.970783,6.069300e+07,7.927576,0.736815,0.866025,5.046425e+08,...,9.332485,21.995588,1015.527027,1365.720819,24.042928,19.336917,76.651664,1.913664,0.770767,1223.466000
3,2011,0.720537,7.032342e+07,3.655681e+06,16.962954,6.083140e+07,7.917310,0.736280,0.867397,4.854622e+08,...,10.985689,22.281141,1015.826453,1364.947801,24.010681,19.358528,76.880381,1.873595,0.771775,1271.961000
4,2012,0.722652,7.118644e+07,3.633845e+06,16.845395,6.065703e+07,9.395868,0.735628,0.868109,4.867661e+08,...,8.159014,22.086723,1016.012434,1378.888334,24.078912,19.469132,77.131969,1.824031,0.772417,1354.740000
5,2013,0.725207,7.173565e+07,3.582357e+06,16.692965,6.034149e+07,9.956265,0.736097,0.868535,5.225848e+08,...,7.913561,21.718626,1016.259286,1398.505820,24.204632,19.429450,76.502389,1.816378,0.773619,1378.303000
6,2014,0.727224,7.218048e+07,3.534361e+06,16.602277,6.008490e+07,10.450155,0.736271,0.868413,5.676413e+08,...,10.350442,20.943935,1016.338200,1416.529798,24.555390,19.517780,75.430446,1.825872,0.774337,1296.041857
7,2015,0.728742,7.217382e+07,3.504012e+06,16.477049,5.965484e+07,10.942052,0.735794,0.867781,5.179392e+08,...,12.454398,20.016361,1016.401227,1398.065883,24.641173,19.598871,75.258234,1.793707,0.774532,1250.323886
8,2016,0.735711,7.236241e+07,3.485388e+06,16.442296,5.953784e+07,10.979229,0.731163,0.879440,4.524144e+08,...,13.111908,19.950370,1016.507582,1365.698492,24.390776,19.578235,75.629199,1.728129,0.778991,1210.917257
9,2017,0.736049,7.274917e+07,3.492937e+06,16.497126,5.973414e+07,10.448735,0.731541,0.878752,4.213689e+08,...,9.479812,20.563357,1016.418014,1341.552229,24.409722,19.738208,76.145059,1.661760,0.778991,1097.369000


In [32]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Espírito Santo - IDH Educacao,Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - Desemprego,Espírito Santo - IDH Renda,Espírito Santo - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Espírito Santo - IDH
0,0.712893,6.711793e+07,3.666068e+06,16.994002,6.044984e+07,7.934578,0.738517,0.861639,5.483335e+08,2.546644e+09,...,2.868108,11.760983,21.747715,1015.304010,1377.758492,24.341274,19.583877,76.458036,1.982089,0.768383
1,0.715558,6.738295e+07,3.683277e+06,16.935445,6.029388e+07,7.964300,0.737559,0.864109,5.302764e+08,2.679945e+09,...,2.305885,9.472585,22.032901,1015.268312,1389.444333,24.312317,19.520664,76.321047,1.962288,0.769638
2,0.718108,6.901541e+07,3.673390e+06,16.970783,6.069300e+07,7.927576,0.736815,0.866025,5.046425e+08,2.891930e+09,...,1.958246,9.332485,21.995588,1015.527027,1365.720819,24.042928,19.336917,76.651664,1.913664,0.770767
3,0.720537,7.032342e+07,3.655681e+06,16.962954,6.083140e+07,7.917310,0.736280,0.867397,4.854622e+08,3.091988e+09,...,2.079088,10.985689,22.281141,1015.826453,1364.947801,24.010681,19.358528,76.880381,1.873595,0.771775
4,0.722652,7.118644e+07,3.633845e+06,16.845395,6.065703e+07,9.395868,0.735628,0.868109,4.867661e+08,3.189506e+09,...,2.183282,8.159014,22.086723,1016.012434,1378.888334,24.078912,19.469132,77.131969,1.824031,0.772417
5,0.725207,7.173565e+07,3.582357e+06,16.692965,6.034149e+07,9.956265,0.736097,0.868535,5.225848e+08,3.093872e+09,...,2.403445,7.913561,21.718626,1016.259286,1398.505820,24.204632,19.429450,76.502389,1.816378,0.773619
6,0.727224,7.218048e+07,3.534361e+06,16.602277,6.008490e+07,10.450155,0.736271,0.868413,5.676413e+08,2.779340e+09,...,2.748397,10.350442,20.943935,1016.338200,1416.529798,24.555390,19.517780,75.430446,1.825872,0.774337
7,0.728742,7.217382e+07,3.504012e+06,16.477049,5.965484e+07,10.942052,0.735794,0.867781,5.179392e+08,2.561731e+09,...,2.501529,12.454398,20.016361,1016.401227,1398.065883,24.641173,19.598871,75.258234,1.793707,0.774532
8,0.735711,7.236241e+07,3.485388e+06,16.442296,5.953784e+07,10.979229,0.731163,0.879440,4.524144e+08,2.516859e+09,...,2.073464,13.111908,19.950370,1016.507582,1365.698492,24.390776,19.578235,75.629199,1.728129,0.778991
9,0.736049,7.274917e+07,3.492937e+06,16.497126,5.973414e+07,10.448735,0.731541,0.878752,4.213689e+08,2.497710e+09,...,2.177386,9.479812,20.563357,1016.418014,1341.552229,24.409722,19.738208,76.145059,1.661760,0.778991


In [33]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     1138.824000
1     1223.466000
2     1271.961000
3     1354.740000
4     1378.303000
5     1296.041857
6     1250.323886
7     1210.917257
8     1097.369000
9     1146.390000
10    1139.906000
11    1138.655000
12    1135.022000
13    1040.260000
14            NaN
Name: Espírito Santo - Consumo de Cimento (t), dtype: float64

In [34]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Espírito Santo - IDH Educacao,Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - Desemprego,Espírito Santo - IDH Renda,Espírito Santo - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Espírito Santo - IDH
0,-1.640005,-1.953465,1.158112,1.368579,0.672334,-1.335550,1.433523,-1.507819,1.204752,-0.990234,...,2.026352,0.984445,0.597392,-1.709857,0.278899,-0.030893,0.246757,0.487054,1.526433,-1.598823
1,-1.312980,-1.818740,1.388817,1.086694,0.303388,-1.310625,1.049506,-1.077403,0.820657,-0.546688,...,0.085558,-0.051710,0.961667,-1.793773,0.675417,-0.165351,-0.212494,0.248808,1.358415,-1.254036
2,-1.000113,-0.988861,1.256278,1.256809,1.247587,-1.341422,0.751226,-0.743733,0.275392,0.158672,...,-1.114490,-0.115145,0.914006,-1.185613,-0.129558,-1.416236,-1.547425,0.823805,0.945813,-0.943799
3,-0.702090,-0.323921,1.018859,1.219118,1.575013,-1.350031,0.536767,-0.504668,-0.132596,0.824346,...,-0.697344,0.633403,1.278750,-0.481752,-0.155787,-1.565973,-1.390424,1.221579,0.605798,-0.666831
4,-0.442517,0.114804,0.726130,0.653201,1.162492,-0.110126,0.275130,-0.380594,-0.104860,1.148829,...,-0.337667,-0.646476,1.030415,-0.044567,0.317236,-1.249149,-0.586872,1.659132,0.185220,-0.490262
5,-0.129103,0.393999,0.035892,-0.080580,0.416019,0.359817,0.463137,-0.306424,0.657046,0.830615,...,0.422337,-0.757614,0.560236,0.535708,0.982887,-0.665378,-0.875166,0.564191,0.120284,-0.160140
6,0.118406,0.620134,-0.607552,-0.517141,-0.190997,0.773988,0.533180,-0.327692,1.615453,-0.215960,...,1.613108,0.345772,-0.429294,0.721210,1.594468,0.963338,-0.233444,-1.300090,0.200839,0.037208
7,0.304741,0.616746,-1.014403,-1.119978,-1.208405,1.186488,0.341797,-0.437835,0.558230,-0.940033,...,0.760923,1.298413,-1.614106,0.869367,0.967960,1.361663,0.355690,-1.599593,-0.072094,0.090719
8,1.159813,0.712619,-1.264069,-1.287275,-1.485208,1.217665,-1.514845,1.593253,-0.835563,-1.089341,...,-0.716758,1.596124,-1.698399,1.119376,-0.130315,0.198966,0.205765,-0.954425,-0.628565,1.316096
9,1.201211,0.909233,-1.162874,-1.023327,-1.020806,0.772798,-1.363136,1.473523,-1.495939,-1.153056,...,-0.358019,-0.048438,-0.915416,0.908829,-0.949634,0.286943,1.367985,-0.057264,-1.191742,1.315880


In [35]:
reshaped_train_input = dfToInputRNN(train_input)

In [36]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[ 1.09097418,  0.70023783, -0.58512371, -0.79832911,
         -0.83501949,  0.52156266, -1.07678861,  0.94091275,
         -1.0453709 ,  1.21859064, -0.99976018,  0.79428371,
          1.15575749, -0.52559858,  1.30260559, -0.55253046,
         -0.25130359,  1.38787176,  1.47154451, -1.80649616,
          1.09296291,  0.94281138,  0.52836084,  0.61258103,
          0.87656257,  1.47801536,  1.71982086,  1.28221181,
          1.39086472,  1.58209844,  1.08872091,  0.55212962,
         -1.81398118,  1.44113674,  1.11191094,  1.30470835,
          0.86390516, -1.67365968, -1.70738939, -1.55813196,
         -0.48498301, -0.75714639,  0.40770893, -0.27394734,
          2.02323219,  1.2908951 ,  1.70465513,  1.81148084,
          1.39968653, -1.36191442,  1.28024593,  1.11054795,
          0.12662799,  1.40414112,  1.40468999,  1.45643959,
          1.58509336,  1.32283157,  1.30222047,  1.55138602,
          1.91801602, -0.77489784,  2.32098884, -1.1942541 ,
         -1.17081042, -0

In [37]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0     1138.824000
1     1223.466000
2     1271.961000
3     1354.740000
4     1378.303000
5     1296.041857
6     1250.323886
7     1210.917257
8     1097.369000
9     1146.390000
10    1139.906000
11    1138.655000
Name: Espírito Santo - Consumo de Cimento (t), dtype: float64

In [38]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    1138.655
Name: Espírito Santo - Consumo de Cimento (t), dtype: float64

In [39]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Espírito Santo - IDH Educacao,Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - Desemprego,Espírito Santo - IDH Renda,Espírito Santo - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Espírito Santo - IDH
12,1.034661,0.301229,-0.384884,-0.769809,-0.984529,0.413158,-1.592938,0.513487,1.108405,2.635823,...,-1.482779,-2.260447,-1.629472,0.114112,-2.051199,0.386249,-0.879822,-1.267572,-1.512676,0.651496


In [40]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[ 1.0346612 ,  0.30122896, -0.38488367, -0.76980913,
         -0.98452948,  0.4131582 , -1.59293772,  0.51348734,
          1.10840491,  2.63582309, -0.18995911,  0.72640489,
          2.66833308, -0.032632  ,  0.98802435,  0.24980494,
         -0.08955808,  1.85434394,  2.02706568, -2.21150703,
          1.42938591,  0.85132668,  0.14092386,  0.30902324,
          1.00267904,  2.27970137,  1.11272593,  0.61007902,
          0.89064538,  1.42231438,  1.02650562,  1.47127593,
         -1.53201473,  1.73503858,  1.66610613,  1.0456894 ,
          0.55289259, -1.81216593, -2.10342296, -2.55377478,
          0.41940628, -1.75756445,  0.241023  , -0.98659345,
          1.52184916,  1.01398607,  1.49290967,  1.40764092,
          1.87930411, -2.51892072,  2.83356214,  1.58620924,
         -0.02524128,  1.54231542,  1.5176119 ,  1.7355269 ,
          1.87941982,  1.54198129,  1.51276712,  1.83795594,
          2.62210693,  1.04993174,  0.96476493, -1.48277861,
         -2.26044738, -1

In [41]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    1135.022
Name: Espírito Santo - Consumo de Cimento (t), dtype: float64

In [53]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=750, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True)
    
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh', return_sequences=True)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [54]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [55]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3348414201, 3553699104, 2513779600, 578400058, 1704213767, 1295350123, 2294176706, 1831001952, 2706042446, 1455176865, 315943488, 406174053, 3545231609, 2209502845, 751019279, 3146597370, 753004993, 4171535732, 2676029328, 1944495025, 1420212095, 2997263470, 4261488299, 2760477227, 3027581853, 2836608879, 110355345, 709907428, 1147449286, 2728984263, 949034377, 672872101, 2350388153, 2777578830, 2533790494, 739091961, 131029519, 111972921, 467756460, 1378329898, 1740912482, 3834669843, 2759979657, 4098898329, 2524313407, 285794154, 538535211, 710987702, 1357015730, 903366620, 1922608869, 2107866929, 2515313829, 2769019773, 660365054, 622161437, 1832579752, 2114581174, 1273371122, 2030899287, 247705889, 3037505674, 3923382155, 945397687, 2157559122, 3929706176, 3162372503, 1433460203, 56591201, 2964771092, 3114845055, 3568794796, 2713819407, 1433724181, 1590449023, 3096024383, 3291398699, 1536966946, 2371564320, 333369499, 913226962, 373460567, 1744349435, 810528170, 651233672, 3234223

loss: 465.0896911621094


Step: 89 ___________________________________________
loss: 637.3729248046875


Step: 90 ___________________________________________
loss: 740.83056640625


Step: 91 ___________________________________________
loss: 664.7274780273438


Step: 92 ___________________________________________
loss: 415.84619140625


Step: 93 ___________________________________________
loss: 448.0674743652344


Step: 94 ___________________________________________
loss: 1044.3472900390625


Step: 95 ___________________________________________
loss: 334.1412658691406


Step: 96 ___________________________________________
loss: 481.42724609375


Step: 97 ___________________________________________
loss: 525.25146484375


Step: 98 ___________________________________________
loss: 536.9217529296875


Step: 99 ___________________________________________
loss: 439.8973388671875


final_seed: 1704213767


In [56]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 2s 2s/step - loss: 1497549.1250
Epoch 2/10000
1/1 [==============================] - 0s 8ms/step - loss: 1463776.3750
Epoch 3/10000
1/1 [==============================] - 0s 8ms/step - loss: 1303114.1250
Epoch 4/10000
1/1 [==============================] - 0s 7ms/step - loss: 302412.7188
Epoch 5/10000
1/1 [==============================] - 0s 7ms/step - loss: 66452.2812
Epoch 6/10000
1/1 [==============================] - 0s 7ms/step - loss: 39074.5273
Epoch 7/10000
1/1 [==============================] - 0s 8ms/step - loss: 25111.4004
Epoch 8/10000
1/1 [==============================] - 0s 8ms/step - loss: 10486.0195
Epoch 9/10000
1/1 [==============================] - 0s 8ms/step - loss: 7409.3774
Epoch 10/10000
1/1 [==============================] - 0s 4ms/step - loss: 10003.7197
Epoch 11/10000
1/1 [==============================] - 0s 5ms/step - loss: 7463.9727
Epoch 12/10000
1/1 [==============================] - 0s 4ms/step - lo

1/1 [==============================] - 0s 7ms/step - loss: 8509.7705
Epoch 99/10000
1/1 [==============================] - 0s 6ms/step - loss: 7250.7017
Epoch 100/10000
1/1 [==============================] - 0s 6ms/step - loss: 4290.8281
Epoch 101/10000
1/1 [==============================] - 0s 5ms/step - loss: 7061.1890
Epoch 102/10000
1/1 [==============================] - 0s 5ms/step - loss: 7984.7915
Epoch 103/10000
1/1 [==============================] - 0s 5ms/step - loss: 5990.7388
Epoch 104/10000
1/1 [==============================] - 0s 5ms/step - loss: 5306.8618
Epoch 105/10000
1/1 [==============================] - 0s 4ms/step - loss: 7570.1392
Epoch 106/10000
1/1 [==============================] - 0s 4ms/step - loss: 3402.6536
Epoch 107/10000
1/1 [==============================] - 0s 5ms/step - loss: 4929.1362
Epoch 108/10000
1/1 [==============================] - 0s 4ms/step - loss: 4058.7971
Epoch 109/10000
1/1 [==============================] - 0s 4ms/step - loss: 4951.20

1/1 [==============================] - 0s 5ms/step - loss: 3558.2395
Epoch 195/10000
1/1 [==============================] - 0s 4ms/step - loss: 4241.1475
Epoch 196/10000
1/1 [==============================] - 0s 5ms/step - loss: 1668.9113
Epoch 197/10000
1/1 [==============================] - 0s 4ms/step - loss: 3841.9941
Epoch 198/10000
1/1 [==============================] - 0s 4ms/step - loss: 1625.7728
Epoch 199/10000
1/1 [==============================] - 0s 5ms/step - loss: 2849.6140
Epoch 200/10000
1/1 [==============================] - 0s 4ms/step - loss: 1702.9852
Epoch 201/10000
1/1 [==============================] - 0s 4ms/step - loss: 4067.4915
Epoch 202/10000
1/1 [==============================] - 0s 5ms/step - loss: 3599.1868
Epoch 203/10000
1/1 [==============================] - 0s 6ms/step - loss: 4064.3879
Epoch 204/10000
1/1 [==============================] - 0s 4ms/step - loss: 5934.8306
Epoch 205/10000
1/1 [==============================] - 0s 6ms/step - loss: 3209.2

1/1 [==============================] - 0s 4ms/step - loss: 2366.5750
Epoch 291/10000
1/1 [==============================] - 0s 5ms/step - loss: 2648.7646
Epoch 292/10000
1/1 [==============================] - 0s 5ms/step - loss: 5170.7705
Epoch 293/10000
1/1 [==============================] - 0s 5ms/step - loss: 3221.8562
Epoch 294/10000
1/1 [==============================] - 0s 4ms/step - loss: 1819.7479
Epoch 295/10000
1/1 [==============================] - 0s 4ms/step - loss: 5310.1851
Epoch 296/10000
1/1 [==============================] - 0s 5ms/step - loss: 2481.7812
Epoch 297/10000
1/1 [==============================] - 0s 5ms/step - loss: 2666.5547
Epoch 298/10000
1/1 [==============================] - 0s 4ms/step - loss: 3698.8821
Epoch 299/10000
1/1 [==============================] - 0s 5ms/step - loss: 2786.1777
Epoch 300/10000
1/1 [==============================] - 0s 5ms/step - loss: 3411.6748
Epoch 301/10000
1/1 [==============================] - 0s 4ms/step - loss: 1434.7

1/1 [==============================] - 0s 7ms/step - loss: 3951.4688
Epoch 387/10000
1/1 [==============================] - 0s 5ms/step - loss: 1945.2357
Epoch 388/10000
1/1 [==============================] - 0s 5ms/step - loss: 1483.3511
Epoch 389/10000
1/1 [==============================] - 0s 5ms/step - loss: 3713.6348
Epoch 390/10000
1/1 [==============================] - 0s 7ms/step - loss: 1422.4171
Epoch 391/10000
1/1 [==============================] - 0s 6ms/step - loss: 2669.6101
Epoch 392/10000
1/1 [==============================] - 0s 7ms/step - loss: 1757.2128
Epoch 393/10000
1/1 [==============================] - 0s 6ms/step - loss: 2924.7295
Epoch 394/10000
1/1 [==============================] - 0s 4ms/step - loss: 3108.1816
Epoch 395/10000
1/1 [==============================] - 0s 7ms/step - loss: 1913.4271
Epoch 396/10000
1/1 [==============================] - 0s 5ms/step - loss: 3688.8679
Epoch 397/10000
1/1 [==============================] - 0s 5ms/step - loss: 2518.5

...

1/1 [==============================] - 0s 4ms/step - loss: 3007.8896
Epoch 867/10000
1/1 [==============================] - 0s 5ms/step - loss: 6562.0757
Epoch 868/10000
1/1 [==============================] - 0s 5ms/step - loss: 1908.3569
Epoch 869/10000
1/1 [==============================] - 0s 5ms/step - loss: 6201.9219
Epoch 870/10000
1/1 [==============================] - 0s 5ms/step - loss: 4863.0005
Epoch 871/10000
1/1 [==============================] - 0s 4ms/step - loss: 3126.8789
Epoch 872/10000
1/1 [==============================] - 0s 4ms/step - loss: 2815.9036
Epoch 873/10000
1/1 [==============================] - 0s 4ms/step - loss: 3578.0994
Epoch 874/10000
1/1 [==============================] - 0s 5ms/step - loss: 3569.9707
Epoch 875/10000
1/1 [==============================] - 0s 5ms/step - loss: 2225.7100
Epoch 876/10000
1/1 [==============================] - 0s 4ms/step - loss: 2312.0176
Epoch 877/10000
1/1 [==============================] - 0s 5ms/step - loss: 4115.0

1/1 [==============================] - 0s 4ms/step - loss: 3566.2190
Epoch 963/10000
1/1 [==============================] - 0s 5ms/step - loss: 8222.7158
Epoch 964/10000
1/1 [==============================] - 0s 4ms/step - loss: 3257.9424
Epoch 965/10000
1/1 [==============================] - 0s 4ms/step - loss: 4663.6235
Epoch 966/10000
1/1 [==============================] - 0s 4ms/step - loss: 2010.6510
Epoch 967/10000
1/1 [==============================] - 0s 4ms/step - loss: 2646.8113
Epoch 968/10000
1/1 [==============================] - 0s 4ms/step - loss: 5645.8696
Epoch 969/10000
1/1 [==============================] - 0s 5ms/step - loss: 4429.2363
Epoch 970/10000
1/1 [==============================] - 0s 5ms/step - loss: 3807.4011
Epoch 971/10000
1/1 [==============================] - 0s 6ms/step - loss: 5367.4185
Epoch 972/10000
1/1 [==============================] - 0s 5ms/step - loss: 2967.8171
Epoch 973/10000
1/1 [==============================] - 0s 5ms/step - loss: 1030.7

1/1 [==============================] - 0s 4ms/step - loss: 3390.8972
Epoch 1058/10000
1/1 [==============================] - 0s 4ms/step - loss: 3789.9207
Epoch 1059/10000
1/1 [==============================] - 0s 5ms/step - loss: 2894.8821
Epoch 1060/10000
1/1 [==============================] - 0s 5ms/step - loss: 4270.1060
Epoch 1061/10000
1/1 [==============================] - 0s 5ms/step - loss: 1352.5236
Epoch 1062/10000
1/1 [==============================] - 0s 5ms/step - loss: 3761.4639
Epoch 1063/10000
1/1 [==============================] - 0s 4ms/step - loss: 3352.4954
Epoch 1064/10000
1/1 [==============================] - 0s 5ms/step - loss: 3262.6848
Epoch 1065/10000
1/1 [==============================] - 0s 5ms/step - loss: 2976.3860
Epoch 1066/10000
1/1 [==============================] - 0s 4ms/step - loss: 3504.2708
Epoch 1067/10000
1/1 [==============================] - 0s 4ms/step - loss: 2470.3071
Epoch 1068/10000
1/1 [==============================] - 0s 5ms/step - l

1/1 [==============================] - 0s 5ms/step - loss: 2894.7146
Epoch 1153/10000
1/1 [==============================] - 0s 5ms/step - loss: 2249.3923
Epoch 1154/10000
1/1 [==============================] - 0s 5ms/step - loss: 2514.1416
Epoch 1155/10000
1/1 [==============================] - 0s 5ms/step - loss: 2286.7454
Epoch 1156/10000
1/1 [==============================] - 0s 5ms/step - loss: 1246.1041
Epoch 1157/10000
1/1 [==============================] - 0s 4ms/step - loss: 1603.6027
Epoch 1158/10000
1/1 [==============================] - 0s 4ms/step - loss: 992.2781
Epoch 1159/10000
1/1 [==============================] - 0s 5ms/step - loss: 4985.1792
Epoch 1160/10000
1/1 [==============================] - 0s 6ms/step - loss: 2466.0356
Epoch 1161/10000
1/1 [==============================] - 0s 5ms/step - loss: 2957.8191
Epoch 1162/10000
1/1 [==============================] - 0s 6ms/step - loss: 3388.8269
Epoch 1163/10000
1/1 [==============================] - 0s 6ms/step - lo

1/1 [==============================] - 0s 4ms/step - loss: 2916.5020
Epoch 1248/10000
1/1 [==============================] - 0s 6ms/step - loss: 2933.2209
Epoch 1249/10000
1/1 [==============================] - 0s 5ms/step - loss: 3081.5066
Epoch 1250/10000
1/1 [==============================] - 0s 6ms/step - loss: 4557.7593
Epoch 1251/10000
1/1 [==============================] - 0s 5ms/step - loss: 3702.3848
Epoch 1252/10000
1/1 [==============================] - 0s 6ms/step - loss: 5671.2056
Epoch 1253/10000
1/1 [==============================] - 0s 5ms/step - loss: 1771.3202
Epoch 1254/10000
1/1 [==============================] - 0s 6ms/step - loss: 2762.3909
Epoch 1255/10000
1/1 [==============================] - 0s 6ms/step - loss: 3993.9141
Epoch 1256/10000
1/1 [==============================] - 0s 5ms/step - loss: 1090.4137
Epoch 1257/10000
1/1 [==============================] - 0s 4ms/step - loss: 2569.2334
Epoch 1258/10000
1/1 [==============================] - 0s 6ms/step - l

In [57]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 558ms/step
[1135.022](test_target) - [[1136.9034]](prediction) = 1.8814423828125655


In [58]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.0016576263568570174

In [59]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [60]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Espírito Santo - IDH Educacao,Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - Desemprego,Espírito Santo - IDH Renda,Espírito Santo - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Espírito Santo - IDH
0,-1.233676,-0.859136,-1.159638,1.132190,-0.177000,-0.475704,1.272700,-1.273039,1.148205,-1.122043,...,1.308949,1.412346,-1.403933,-0.545434,0.012109,0.812140,0.987054,-0.139198,1.022953,-1.245701
1,0.018063,-0.543270,1.280840,-1.299995,-1.126614,1.391229,-0.102332,0.103108,0.140893,-0.184474,...,-0.190804,-0.643250,0.849376,-0.857273,1.218646,0.596588,0.383567,-1.149199,0.334205,0.043048
2,1.215613,1.402406,-0.121202,0.167805,1.303615,-0.915525,-1.170368,1.169932,-1.289098,1.306518,...,-1.118144,-0.769096,0.554557,1.402706,-1.230754,-1.408727,-1.370621,1.288397,-1.357157,1.202653


train_target:


,Espírito Santo - Consumo de Cimento (t)
0,1138.824
1,1223.466
2,1271.961


test_input:


...

,Espírito Santo - IDH Educacao,Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - Desemprego,Espírito Santo - IDH Renda,Espírito Santo - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Espírito Santo - IDH
12,1.034661,0.301229,-0.384884,-0.769809,-0.984529,0.413158,-1.592938,0.513487,1.108405,2.635823,...,-1.482779,-2.260447,-1.629472,0.114112,-2.051199,0.386249,-0.879822,-1.267572,-1.512676,0.651496


test_target:


,Espírito Santo - Consumo de Cimento (t)
12,1135.022


1/1 [==============================] - 1s 576ms/step
Error: 1.8814423828125655




[1238.1505126953125,
 1320.0863037109375,
 1148.917724609375,
 1162.886474609375,
 1204.556884765625,
 1236.2515869140625,
 1112.4576416015625,
 1135.941650390625,
 1135.378662109375,
 1136.9034423828125]

In [61]:
display(targets)
display(predictions)

[1354.74,
 1378.303,
 1296.041857142857,
 1250.3238857142856,
 1210.9172571428571,
 1097.369,
 1146.39,
 1139.906,
 1138.655,
 1135.022]

[1238.1505126953125,
 1320.0863037109375,
 1148.917724609375,
 1162.886474609375,
 1204.556884765625,
 1236.2515869140625,
 1112.4576416015625,
 1135.941650390625,
 1135.378662109375,
 1136.9034423828125]

In [62]:
mae = mean_absolute_error(predictions, targets)
mae

59.76651748046875

In [63]:
porcentage = mae/np.mean(targets)
porcentage

0.049199992525700204